# Windowing in Spark

In [17]:
import pyspark
from pyspark.sql import SparkSession
# Create a Spark session
spark = SparkSession.builder.appName("GMMClustering").getOrCreate()
df = spark.createDataFrame(
 [("sales",1,7000),("sales",2,6000),("sales",9,5000),("sales",10,6000),("hr",3,2000),("hr",4,6000),("hr",7,3000),("it",5,5000),("it",6,11000),("it",8,19000)],["dept_id","emp_id","salary"])
df.show()

+-------+------+------+
|dept_id|emp_id|salary|
+-------+------+------+
|  sales|     1|  7000|
|  sales|     2|  6000|
|  sales|     9|  5000|
|  sales|    10|  6000|
|     hr|     3|  2000|
|     hr|     4|  6000|
|     hr|     7|  3000|
|     it|     5|  5000|
|     it|     6| 11000|
|     it|     8| 19000|
+-------+------+------+



### Problem statement: How to calculate a new column for each group whose row value is equal to sum of the current row and previous 2 rows?


In [22]:
from pyspark.sql import functions as func
from pyspark.sql import Window
window = Window.partitionBy("dept_id").orderBy("emp_id").rowsBetween(-2, 0)
df.withColumn("sum",func.sum("salary").over(window)).show()

+-------+------+------+-----+
|dept_id|emp_id|salary|  sum|
+-------+------+------+-----+
|     hr|     3|  2000| 2000|
|     hr|     4|  6000| 8000|
|     hr|     7|  3000|11000|
|     it|     5|  5000| 5000|
|     it|     6| 11000|16000|
|     it|     8| 19000|35000|
|  sales|     1|  7000| 7000|
|  sales|     2|  6000|13000|
|  sales|     9|  5000|18000|
|  sales|    10|  6000|17000|
+-------+------+------+-----+



In [24]:
window = Window.partitionBy("dept_id").orderBy("emp_id").rowsBetween(0, 2)
df.withColumn("sum",func.sum("salary").over(window)).show()


+-------+------+------+-----+
|dept_id|emp_id|salary|  sum|
+-------+------+------+-----+
|     hr|     3|  2000|11000|
|     hr|     4|  6000| 9000|
|     hr|     7|  3000| 3000|
|     it|     5|  5000|35000|
|     it|     6| 11000|30000|
|     it|     8| 19000|19000|
|  sales|     1|  7000|18000|
|  sales|     2|  6000|17000|
|  sales|     9|  5000|11000|
|  sales|    10|  6000| 6000|
+-------+------+------+-----+



### How to calculate the lag


In [25]:
window = Window.partitionBy("dept_id").orderBy(func.col("salary").desc())
df.withColumn("previousrow_salary",func.lag('salary',1).over(window)).show()


+-------+------+------+------------------+
|dept_id|emp_id|salary|previousrow_salary|
+-------+------+------+------------------+
|     hr|     4|  6000|              NULL|
|     hr|     7|  3000|              6000|
|     hr|     3|  2000|              3000|
|     it|     8| 19000|              NULL|
|     it|     6| 11000|             19000|
|     it|     5|  5000|             11000|
|  sales|     1|  7000|              NULL|
|  sales|     2|  6000|              7000|
|  sales|    10|  6000|              6000|
|  sales|     9|  5000|              6000|
+-------+------+------+------------------+

